# How to stream runnables
Streaming is critical in making applications based on LLMs feel responsive to end-users. This interface provider 2 general approches:
1. sync `stream` and async `astream`: a default implementation of streaming that streams the final output from the chain.
2. async `astream_events` and async `astream_log`: these provide a way to stream both intermediate steps and final output from the chain.

In [3]:
from langchain_community.llms import Ollama
model = Ollama(model="llama3")

# Using Stream
NOTE: select from working environment it meant if model streaming run on async function, you should use `astream` API

In [10]:
# sync stream API:
chunks = []
for chunk in model.stream("what color is the sky? Answer in short sentence."):
  chunks.append(chunk)
  print(chunk, end="|", flush=True)

The| sky| appears| blue| to| our| eyes| during| the| daytime|,| but| it| can| take| on| hues| of| red|,| orange|,| and| purple| during| sunrise| and| sunset|.||

In [11]:
# async astream API:
chunks = []
async for chunk in model.astream("what color is the sky? Answer in short sentence."):
  chunks.append(chunk)
  print(chunk, end="|", flush=True)
  # print(chuck.content, end="|", flush=True) # depending on models

The| sky| appears| blue| to| our| eyes|,| but| it| can| also| appear| other| colors| depending| on| the| time| of| day| and| atmospheric| conditions|.||

In [12]:
chunks[0]
# Some model, you can got back something called AIMessageChunk
# from langchain_core.messages.ai import AIMessageChunk
# AIMessageChunk(content="The", id='run-b36bea64-5511-4d7a-b6a3-a07b3db0c8e7')

'The'

In [16]:
# Chains
# involves more step using LangChain Expression Language (LCEL): combines a prompt, model and a parser and verify that streaming works
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

Here|'s| one|:

|Why| did| the| par|rot| go| to| the| doctor|?

|Because| it| had| a| f|owl| temper|!

|Hope| that| made| you| squ|awk| with| laughter|!||

In [17]:
from langchain_core.output_parsers import JsonOutputParser

chain = model | JsonOutputParser()  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models
async for text in chain.astream(
  "output a list of the countries france, spain and japan and their populations in JSON format. "
  'Use a dict with an outer key of "countries" which contains a list of countries. '
  "Each country should have the key `name` and `population`"
):
  print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 653}]}
{'countries': [{'name': 'France', 'population': 653451}]}
{'countries': [{'name': 'France', 'population': 65345100}]}
{'countries': [{'name': 'France', 'population': 65345100}, {}]}
{'countries': [{'name': 'France', 'population': 65345100}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65345100}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 65345100}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65345100}, {'name': 'Spain', 'population': 467527}]}
{'countries': [{'name': 'France', 'population': 65345100}, {'name': 'Spain', 'population': 46752716}]}
{'countries': [{'name': 'France', 'population': 65345100}, {'name': 'Spain', 'population': 46752716}, {}]}
{'countries': [{'name': 'France', 'population': 65345100}, {'name': 'Spain', 'population': 467

In [18]:
# Any steps in the chain that operate on finalized inputs rather than on input streams can break streaming functionality via stream or astream.
from langchain_core.output_parsers import JsonOutputParser

# A function that operates on finalized inputs
# Streaming will not work with this function
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""
    if "countries" not in inputs:
        return ""
    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""
    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

In [19]:
# Solve the problem using generator functions
# generator functions (yield): a function that returns an iterator and can be paused and resumed
from langchain_core.output_parsers import JsonOutputParser

async def _extract_country_names_streaming(input_stream):
  """A function that operates on input streams."""
  country_names_so_far = set()

  async for input in input_stream:
    if not isinstance(input, dict):
      continue

    if "countries" not in input:
      continue

    countries = input["countries"]

    if not isinstance(countries, list):
      continue

    for country in countries:
      name = country.get("name")
      if not name:
        continue
      if name not in country_names_so_far:
        yield name
        country_names_so_far.add(name)

chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
  "output a list of the countries france, spain and japan and their populations in JSON format. "
  'Use a dict with an outer key of "countries" which contains a list of countries. '
  "Each country should have the key `name` and `population`",
):
  print(text, end="|", flush=True)

France|Spain|Japan|

In [27]:
# Non-streaming components
# some built-in components like Retriever, which fetches data from the internet, do not support streaming
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
# pip install faiss-gpu or pip install faiss-cpu
vectorstore = FAISS.from_texts(
  ["harrison worked at kensho", "harrison likes spicy food"],
  # embedding=OpenAIEmbeddings(),
  embedding=OllamaEmbeddings(model="llama3"),
)
retriever = vectorstore.as_retriever()
chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(page_content='harrison likes spicy food'),
  Document(page_content='harrison worked at kensho')]]

In [28]:
# But we can use RunnablePassthrough to convert the non-streaming component to a streaming component
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
retrieval_chain = (
  {
    "context": retriever.with_config(run_name="Docs"),
    "question": RunnablePassthrough(),
  }
  | prompt
  | model
  | StrOutputParser()
)
for chunk in retrieval_chain.stream("Where did harrison work? " "Write 3 made up sentences about this place."):
  print(chunk, end="|", flush=True)

Based| on| the| given| context|,| Harrison| worked| at| Kens|ho|.

|Here| are| three| made|-up| sentences| about| Kens|ho|:

|K|ens|ho| is| a| trendy| restaurant| that| serves| an| array| of| international| dishes| with| a| focus| on| bold| flavors| and| spices|.| The| vibrant| atmosphere| and| eclectic| decor| make| it| a| popular| spot| for| food|ies| and| social|ites| alike|.| As| the| go|-to| place| for| spicy| food| enthusiasts|,| Kens|ho|'s| chefs| are| always| experimenting| with| new| hot| sauces| and| marin|ades| to| tantal|ize| taste| buds|.||